# Newon's Method

### Newton's Method

In [85]:
from scipy import linalg as la
import numpy as np

def newton_method(df, d2f, x0, *, tol=10**-6, maxiter=20):
    
    conv = False
    i = 0
    xk = x0
    
    while i < maxiter:
        
        Df = df(xk)
        D2f = d2f(xk)
        zk = la.solve(D2f, np.transpose(Df))
        
        xk = x0 - zk
        
        if la.norm(df(xk), np.inf) < tol:
            conv = True
            break
            
        x0 = xk
        i += 1
        
        
    return xk, conv, i

In [86]:
from scipy import optimize as opt
f = opt.rosen # The Rosenbrock function.
df = opt.rosen_der # The first derivative.
d2f = opt.rosen_hess # The second derivative (Hessian).
x0 = np.array([-2, 2])
newton_method(df, d2f, x0)

(array([1., 1.]), True, 4)

In [87]:
#Optimizing with the built in BFGS function
opt.fmin_bfgs(f=f, x0=[-2,2], fprime=df, maxiter=50)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 35
         Function evaluations: 42
         Gradient evaluations: 42


array([1.00000021, 1.00000045])

### BFGS

In [88]:
def BFGS(df, x0, *, tol=10**-5, maxiter=80):
    
    from numpy.linalg import inv
    
    conv = False
    i = 0
    A0 = np.eye(len(df(x0)))
    
    # Init vals
    xk = x0
    Ak = A0
    
    while i < maxiter:
        
        Df0 = df(xk)
        xk = x0 - inv(Ak) @ np.transpose(Df0)
        Df1 = df(xk)
        sk = xk - x0
        yk = np.transpose(Df1) - np.transpose(Df0)
        syk = sk.T @ yk
        
        Ak = inv(inv(Ak) + ((syk + yk.T @ inv(Ak) @ yk) * np.outer(sk, sk.T))/(syk)**2 \
            - (inv(Ak) @ np.outer(yk, sk.T) + np.outer(sk, yk.T) @ inv(Ak))/syk)
        
        
        if la.norm(Df1, np.inf) < tol:
            conv = True
            break
        
        if syk**2 == 0:
            break
            
        x0 = xk
        Df0 = Df1
        i += 1
        
        
    return xk, conv, i

In [89]:
df = opt.rosen_der # The first derivative.
x0 = np.array([1.5, 1.5])

BFGS(df, x0)

(array([1.00000004, 1.00000008]), True, 33)

### The Gauss-Newton Method

In [90]:
def gauss_newton(f, df, r, dr, x0, *, tol=10**-5, maxiter=10):
    
    conv = False
    i = 0
    xk = x0
    
    while i < maxiter:
        
        Df = df(xk)
        J = dr(xk)
        D2f = J.T @ J
        zk = la.solve(D2f, J.T @ r(xk))
        
        xk = x0 - zk
        
        if la.norm(Df, np.inf) < tol:
            conv = True
            break
            
        x0 = xk
        i += 1
        
        
    return xk, conv, i

In [91]:
# Generate random data for t = 0, 1, ..., 10.
T = np.arange(10)
y = 3*np.sin(0.5*T)+ 0.5*np.random.randn(10) # Perturbed data.
# Define the model function and the residual (based on the data).
model = lambda x, t: x[0]*np.sin(x[1]*t) # phi(x,t)
residual = lambda x: model(x, T) - y # r(x) = phi(x,t) - y
# Define the Jacobian of the residual function, computed by hand.
jac = lambda x: np.column_stack((np.sin(x[1]*T), x[0]*T*np.cos(x[1]*T)))
x0 = np.array([2.5,.6])
gauss_newton(model, jac, residual, jac, x0)

(array([2.89908882, 0.49967729]), False, 10)

In [92]:
minx = opt.leastsq(func=residual, x0=np.array([2.5,.6]), Dfun=jac)

In [93]:
minx

(array([2.89908937, 0.4996771 ]), 1)

### Calculating the parameters of a model using the least squares function

In [94]:
census = np.load('population.npy')

In [95]:
census

array([[  0.   ,   3.929],
       [  1.   ,   5.308],
       [  2.   ,   7.24 ],
       [  3.   ,   9.638],
       [  4.   ,  12.866],
       [  5.   ,  17.069],
       [  6.   ,  23.192],
       [  7.   ,  31.443],
       [  8.   ,  38.558],
       [  9.   ,  50.156],
       [ 10.   ,  62.948],
       [ 11.   ,  75.996],
       [ 12.   ,  91.972],
       [ 13.   , 105.711],
       [ 14.   , 122.775],
       [ 15.   , 131.669]])

In [96]:
T = census[:, 0]
y = census[:, 1]*1_000_000

In [97]:
# Model
model = lambda x, t: x[0]*np.exp(x[1]*(t+x[2]))
residual = lambda x: model(x, T) - y
jac = lambda x: np.column_stack((np.exp(x[1]*(T+x[2])),
                                 x[0]*(T+x[2])*np.exp(x[1]*(T+x[2])),
                                 x[0]*x[1]*np.exp(x[1]*(T+x[2]))))
opt.leastsq(func=residual, x0=np.array([1.5, 4., 2.5]), Dfun=jac)

(array([98.55242473,  0.18474368, 61.85172316]), 1)